# [Module 3.4.1] Train Docker Image 생성 및 ECR 퍼블리시

이 노트북은 Docker 이미지를 생성하고, Amazon ECR(Elastic Container Registry)에 퍼블리스를 함

---
노트북의 소요 시간은 약 2분 걸립니다.

---

Docker Info:
- Get Started: Build Your Custom Training Container with Amazon SageMaker
    - https://docs.aws.amazon.com/sagemaker/latest/dg/build-container-to-train-script-get-started.html
- Built-in Container Image
    - https://github.com/aws/deep-learning-containers/blob/master/available_images.md

## Train Script를 다커 폴더에 복사

In [2]:
bert_train_file = "tf_script_bert_tweet.py"
! cp {bert_train_file} "train_container/"

## train_container 폴더의 Dockerfile 
**해당 Region에 따라 Dockerfile을 수정해야 합니다.**
현재의 Dockerfile은 'ap-northeast-2' 으로 되어 있습니다.
민일 'us-west-2' 이면 
아래와 같이 Dockerfile 의 내용을 바꾸어야 합니다. <br>

FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:2.1.1-gpu-py36-cu101-ubuntu18.04

In [3]:
! pygmentize train_container/Dockerfile


# 기존의  Pre-built-in TF2.1-gpu image를 가져옴
FROM 763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/tensorflow-training:2.1.1-gpu-py36-cu101-ubuntu18.04

# transformers 설치
RUN pip install transformers==2.8.0


ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

ENV PATH="/opt/ml/code:${PATH}"

# Copy training code
COPY tf_script_bert_tweet.py /opt/ml/code/
 
WORKDIR /opt/ml/code

ENV SAGEMAKER_PROGRAM tf_script_bert_tweet.py


# Train Docker 이미지를 ECR (Elastic Container Registry) 에 Push

In [4]:
import os
os.environ['train_container_name']= "bert2tweet"

In [5]:
%%sh
cd train_container

# The name of our algorithm
algorithm_name=$train_container_name


account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

echo $region

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Get the login command from ECR in order to pull down the Tensorflow-gpu:1.5 image
$(aws ecr get-login --registry-ids 763104351884 --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} . --build-arg REGION=${region}
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

ap-northeast-2
Login Succeeded
Login Succeeded
Sending build context to Docker daemon  25.09kB
Step 1/8 : FROM 763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/tensorflow-training:2.1.1-gpu-py36-cu101-ubuntu18.04
2.1.1-gpu-py36-cu101-ubuntu18.04: Pulling from tensorflow-training
7ddbc47eeb70: Pulling fs layer
c1bbdc448b72: Pulling fs layer
8c3b70e39044: Pulling fs layer
45d437916d57: Pulling fs layer
d8f1569ddae6: Pulling fs layer
85386706b020: Pulling fs layer
ee9b457b77d0: Pulling fs layer
8111a252bf4e: Pulling fs layer
e04706919ddc: Pulling fs layer
d8f1569ddae6: Waiting
45d437916d57: Waiting
483096c05cce: Pulling fs layer
eb8bb42cd350: Pulling fs layer
b864935f8c06: Pulling fs layer
bc1f79cc3eb5: Pulling fs layer
ee9b457b77d0: Waiting
8111a252bf4e: Waiting
e04706919ddc: Waiting
eb8bb42cd350: Waiting
85386706b020: Waiting
f50ee009e6e3: Pulling fs layer
bc1f79cc3eb5: Waiting
78d2ec229bc7: Pulling fs layer
69f59377e9f1: Pulling fs layer
f50ee009e6e3: Waiting
7d289e8f9bd8: Pulling fs 

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

